In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models
from torchvision import datasets
from torchvision import transforms
from torchmetrics.classification import MulticlassAccuracy
from torchinfo import summary
import os.path

In [2]:
no_epochs = 50
learning_rate = 0.001
batch_size = 128

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

acc_function = MulticlassAccuracy(num_classes=102, average='micro').to(device)
loss_fn = nn.CrossEntropyLoss()

PATH = './model.pth'

cuda:0


In [18]:
model = models.resnet18()
print(model.fc)

Linear(in_features=512, out_features=1000, bias=True)


In [3]:
# Data Augmentation
train_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Assuming you want to keep the default transformations for testing/validation:
default_transforms = transforms.Compose([
    models.VGG16_BN_Weights.IMAGENET1K_V1.transforms()
])

flowers_train = datasets.Flowers102(root='./data', split='train', download=True, transform=train_transforms)
flowers_test = datasets.Flowers102(root='./data', split='test', download=True, transform=default_transforms)
flowers_val = datasets.Flowers102(root='./data', split='val', download=True, transform=default_transforms)


In [4]:
def get_data_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(flowers_train, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(flowers_test, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(flowers_val, batch_size=batch_size, shuffle=True)
    return train_loader, test_loader, val_loader

In [5]:
# Early stopping based on accuracy
class AccuracyEarlyStopper:
    def __init__(self, patience=3, min_delta=0.5):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.max_validation_accuracy = 0

    def early_stop(self, validation_accuracy):
        if validation_accuracy > (self.max_validation_accuracy + self.min_delta):
            self.max_validation_accuracy = validation_accuracy
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [6]:
def train(model, optimizer, dataloader, loss_fn=loss_fn):
    running_loss_value = 0
    for images, labels in dataloader:
        optimizer.zero_grad()
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        running_loss_value += loss.item()
        loss.backward()
        optimizer.step()
    return running_loss_value / len(dataloader)

def test_eval(model, dataloader, loss_fn=loss_fn):
    running_loss_value = 0
    running_acc_value = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            acc = acc_function(outputs, labels)
            running_loss_value += loss.item()
            running_acc_value += acc.item()
    running_acc_value /= len(dataloader)
    running_loss_value /= len(dataloader)
    return running_acc_value*100, running_loss_value

def train_eval_test(model, train_dataloader, val_dataloader, test_dataloader, no_epochs=10):
    es = AccuracyEarlyStopper()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    train_loss_arr, train_acc_arr, eval_loss_arr, eval_acc_arr = [], [], [], []
    for i in range(no_epochs):
        train_loss = train(model, optimizer, train_dataloader)
        eval_acc, eval_loss = test_eval(model, val_dataloader)
        print(f'Epoch {i+1} Train Loss: {train_loss:>8f}, Eval Accuracy: {eval_acc:>0.2f}%, Eval Loss: {eval_loss:>8f}')
        train_loss_arr.append(train_loss)
        eval_loss_arr.append(eval_loss)
        eval_acc_arr.append(eval_acc)
        if es.early_stop(eval_acc):
            print('Early stopping activated')
            break
    test_acc, test_loss = test_eval(model, test_dataloader)
    print(f"Test Accuracy: {test_acc}, Test Loss: {test_loss}")
    return train_loss_arr, train_acc_arr, eval_loss_arr, eval_acc_arr, test_acc, test_loss

In [7]:
def create_VGG_model():
    model = models.vgg16_bn(weights=models.VGG16_BN_Weights.DEFAULT)
    new_classifier_head = nn.Sequential(
        nn.Linear(25088, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(0.5),
        nn.Linear(4096, 102)
    )
    
    for param in model.parameters():
        param.requires_grad = False
        
    model.classifier = new_classifier_head

    return model.to(device=device)

def create_resnet_model(size=50):
    match size:
        case 18:
            model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
            new_fc = nn.Linear(512,102)
        case 34:
            model = models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
            new_fc = nn.Linear(512,102)
        case 50:
            model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
            new_fc = nn.Linear(2048, 102)
        case 101:
            model = models.resnet101(weights=models.ResNet101_Weights.DEFAULT)
            new_fc = nn.Linear(2048,102)
        case _:
            print("Invalid Size, defaulting to 50")
            model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
            new_fc = nn.Linear(2048, 102)
    
    for param in model.parameters():
        param.requires_grad = False
        
    model.fc = new_fc
    
    return model.to(device)

def create_efficientnet_model(size):
    match size:
        case 's':
            model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)
        case 'm':
            model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
        case 'l':
            model = models.efficientnet_v2_l(weights=models.EfficientNet_V2_L_Weights.DEFAULT)
        case _:
            model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)



    for param in model.parameters():
        param.requires_grad = False
        
    new_classifier = nn.Sequential(
        nn.Dropout(0.2),
        nn.Linear(1280, 102)
    )
    
    model.classifier = new_classifier
    
    return model.to(device)

In [8]:
# if os.path.isfile(PATH):
#     #model.load_state_dict(torch.load(PATH))
#     print("Model loaded")
train_data_loader, test_data_loader, val_data_loader = get_data_loader(batch_size)

In [10]:
VGG_model = create_VGG_model()
print("VGG16")
VGG_train_acc, VGG_train_loss, VGG_eval_acc, VGG_eval_loss, VGG_test_acc, VGG_test_loss = train_eval_test(
    VGG_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

VGG16
Epoch 1 Train Loss: 4.871661, Eval Accuracy: 3.72%, Eval Loss: 4.544444
Epoch 2 Train Loss: 4.502391, Eval Accuracy: 11.74%, Eval Loss: 4.134139
Epoch 3 Train Loss: 3.933663, Eval Accuracy: 25.18%, Eval Loss: 3.229683
Epoch 4 Train Loss: 3.061470, Eval Accuracy: 33.21%, Eval Loss: 2.567947
Epoch 5 Train Loss: 2.485867, Eval Accuracy: 44.61%, Eval Loss: 1.989159
Epoch 6 Train Loss: 1.976284, Eval Accuracy: 53.72%, Eval Loss: 1.663346
Epoch 7 Train Loss: 1.571505, Eval Accuracy: 58.74%, Eval Loss: 1.499054
Epoch 8 Train Loss: 1.442601, Eval Accuracy: 62.14%, Eval Loss: 1.342761
Epoch 9 Train Loss: 1.252551, Eval Accuracy: 66.74%, Eval Loss: 1.191349
Epoch 10 Train Loss: 1.189541, Eval Accuracy: 65.66%, Eval Loss: 1.224204
Epoch 11 Train Loss: 1.019445, Eval Accuracy: 67.71%, Eval Loss: 1.207595
Epoch 12 Train Loss: 0.947347, Eval Accuracy: 69.60%, Eval Loss: 1.153499
Epoch 13 Train Loss: 0.978374, Eval Accuracy: 71.19%, Eval Loss: 1.038815
Epoch 14 Train Loss: 0.904875, Eval Accura

In [11]:
#torch.save(model.state_dict(), PATH)

In [9]:
ResNet_18_model = create_resnet_model(18)
print("RESNET")
ResNet_train_acc, ResNet_rain_loss, ResNet_eval_acc, ResNet_eval_loss, ResNet_test_acc, ResNet_test_loss = train_eval_test(
    ResNet_18_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

RESNET
Epoch 1 Train Loss: 4.797734, Eval Accuracy: 5.00%, Eval Loss: 4.390567
Epoch 2 Train Loss: 4.272619, Eval Accuracy: 16.95%, Eval Loss: 3.992445
Epoch 3 Train Loss: 3.797032, Eval Accuracy: 38.72%, Eval Loss: 3.572695
Epoch 4 Train Loss: 3.413130, Eval Accuracy: 48.23%, Eval Loss: 3.225145
Epoch 5 Train Loss: 3.082503, Eval Accuracy: 60.28%, Eval Loss: 2.891474
Epoch 6 Train Loss: 2.786082, Eval Accuracy: 63.13%, Eval Loss: 2.612772
Epoch 7 Train Loss: 2.471209, Eval Accuracy: 68.34%, Eval Loss: 2.348552
Epoch 8 Train Loss: 2.263146, Eval Accuracy: 72.36%, Eval Loss: 2.148288
Epoch 9 Train Loss: 2.031713, Eval Accuracy: 74.60%, Eval Loss: 1.964003
Epoch 10 Train Loss: 1.826465, Eval Accuracy: 76.68%, Eval Loss: 1.803335
Epoch 11 Train Loss: 1.690033, Eval Accuracy: 79.13%, Eval Loss: 1.673186
Epoch 12 Train Loss: 1.551838, Eval Accuracy: 79.33%, Eval Loss: 1.557030
Epoch 13 Train Loss: 1.439185, Eval Accuracy: 80.70%, Eval Loss: 1.466376
Epoch 14 Train Loss: 1.340897, Eval Accur

In [10]:
ResNet_34_model = create_resnet_model(34)
print("RESNET")
ResNet_train_acc, ResNet_rain_loss, ResNet_eval_acc, ResNet_eval_loss, ResNet_test_acc, ResNet_test_loss = train_eval_test(
    ResNet_34_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

RESNET
Epoch 1 Train Loss: 4.755768, Eval Accuracy: 5.89%, Eval Loss: 4.391149
Epoch 2 Train Loss: 4.207667, Eval Accuracy: 17.92%, Eval Loss: 3.973718
Epoch 3 Train Loss: 3.766797, Eval Accuracy: 34.98%, Eval Loss: 3.561058
Epoch 4 Train Loss: 3.392065, Eval Accuracy: 46.58%, Eval Loss: 3.197053
Epoch 5 Train Loss: 3.011557, Eval Accuracy: 56.76%, Eval Loss: 2.861568
Epoch 6 Train Loss: 2.699158, Eval Accuracy: 63.63%, Eval Loss: 2.573285
Epoch 7 Train Loss: 2.389217, Eval Accuracy: 68.34%, Eval Loss: 2.318256
Epoch 8 Train Loss: 2.185876, Eval Accuracy: 70.68%, Eval Loss: 2.108436
Epoch 9 Train Loss: 1.967856, Eval Accuracy: 72.93%, Eval Loss: 1.931408
Epoch 10 Train Loss: 1.792846, Eval Accuracy: 76.36%, Eval Loss: 1.758007
Epoch 11 Train Loss: 1.671864, Eval Accuracy: 76.86%, Eval Loss: 1.626065
Epoch 12 Train Loss: 1.476047, Eval Accuracy: 79.01%, Eval Loss: 1.515631
Epoch 13 Train Loss: 1.400540, Eval Accuracy: 80.00%, Eval Loss: 1.415168
Epoch 14 Train Loss: 1.231968, Eval Accur

In [ ]:
ResNet_50_model = create_resnet_model(50)
print("RESNET")
ResNet_train_acc, ResNet_rain_loss, ResNet_eval_acc, ResNet_eval_loss, ResNet_test_acc, ResNet_test_loss = train_eval_test(
    ResNet_50_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

RESNET
Epoch 1 Train Loss: 4.546194, Eval Accuracy: 21.66%, Eval Loss: 4.288802
Epoch 2 Train Loss: 4.143304, Eval Accuracy: 52.27%, Eval Loss: 3.952171
Epoch 3 Train Loss: 3.804565, Eval Accuracy: 67.47%, Eval Loss: 3.646060
Epoch 4 Train Loss: 3.468051, Eval Accuracy: 74.03%, Eval Loss: 3.366712
Epoch 5 Train Loss: 3.179319, Eval Accuracy: 77.35%, Eval Loss: 3.089919


In [14]:
ResNet_101_model = create_resnet_model(101)
print("RESNET")
ResNet_train_acc, ResNet_rain_loss, ResNet_eval_acc, ResNet_eval_loss, ResNet_test_acc, ResNet_test_loss = train_eval_test(
    ResNet_101_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /home/UG/chan0965/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth
55.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

73.9%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

95.2%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--Notebook

Epoch 1 Train Loss: 4.570124, Eval Accuracy: 15.90%, Eval Loss: 4.236614
Epoch 2 Train Loss: 4.050102, Eval Accuracy: 36.78%, Eval Loss: 3.826074
Epoch 3 Train Loss: 3.615210, Eval Accuracy: 56.20%, Eval Loss: 3.442326
Epoch 4 Train Loss: 3.249373, Eval Accuracy: 66.86%, Eval Loss: 3.101753
Epoch 5 Train Loss: 2.893960, Eval Accuracy: 71.15%, Eval Loss: 2.800291
Epoch 6 Train Loss: 2.601976, Eval Accuracy: 74.30%, Eval Loss: 2.538348
Epoch 7 Train Loss: 2.306975, Eval Accuracy: 74.81%, Eval Loss: 2.311848
Epoch 8 Train Loss: 2.035474, Eval Accuracy: 75.48%, Eval Loss: 2.136037
Epoch 9 Train Loss: 1.874005, Eval Accuracy: 76.68%, Eval Loss: 1.961802
Epoch 10 Train Loss: 1.667907, Eval Accuracy: 78.43%, Eval Loss: 1.811005
Epoch 11 Train Loss: 1.530197, Eval Accuracy: 78.63%, Eval Loss: 1.690355
Epoch 12 Train Loss: 1.412507, Eval Accuracy: 78.86%, Eval Loss: 1.590276
Epoch 13 Train Loss: 1.251828, Eval Accuracy: 79.91%, Eval Loss: 1.503392
Epoch 14 Train Loss: 1.217869, Eval Accuracy: 8

In [15]:
efficientnet_s_model = create_efficientnet_model('s')

print("EFFICIENTNET")

EN_train_acc, EN_rain_loss, EN_eval_acc, EN_eval_loss, EN_test_acc, EN_test_loss = train_eval_test(
    efficientnet_s_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

EFFICIENTNET
Epoch 1 Train Loss: 4.574721, Eval Accuracy: 12.42%, Eval Loss: 4.353661
Epoch 2 Train Loss: 4.185564, Eval Accuracy: 33.15%, Eval Loss: 4.031021
Epoch 3 Train Loss: 3.860585, Eval Accuracy: 46.29%, Eval Loss: 3.739001
Epoch 4 Train Loss: 3.542766, Eval Accuracy: 53.71%, Eval Loss: 3.490047
Epoch 5 Train Loss: 3.278103, Eval Accuracy: 59.00%, Eval Loss: 3.235458
Epoch 6 Train Loss: 3.031440, Eval Accuracy: 62.44%, Eval Loss: 3.006856
Epoch 7 Train Loss: 2.782584, Eval Accuracy: 66.27%, Eval Loss: 2.804595
Epoch 8 Train Loss: 2.568160, Eval Accuracy: 68.70%, Eval Loss: 2.628160
Epoch 9 Train Loss: 2.379220, Eval Accuracy: 68.03%, Eval Loss: 2.468820
Epoch 10 Train Loss: 2.258689, Eval Accuracy: 69.42%, Eval Loss: 2.334562
Epoch 11 Train Loss: 2.072847, Eval Accuracy: 71.96%, Eval Loss: 2.208270
Epoch 12 Train Loss: 1.962521, Eval Accuracy: 72.17%, Eval Loss: 2.104919
Epoch 13 Train Loss: 1.806893, Eval Accuracy: 72.62%, Eval Loss: 2.006330
Epoch 14 Train Loss: 1.714660, Eva

In [16]:
efficientnet_m_model = create_efficientnet_model('m')

print("EFFICIENTNET")

EN_train_acc, EN_rain_loss, EN_eval_acc, EN_eval_loss, EN_test_acc, EN_test_loss = train_eval_test(
    efficientnet_m_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

EFFICIENTNET
Epoch 1 Train Loss: 4.611644, Eval Accuracy: 7.65%, Eval Loss: 4.419560
Epoch 2 Train Loss: 4.293189, Eval Accuracy: 23.92%, Eval Loss: 4.170458
Epoch 3 Train Loss: 4.053287, Eval Accuracy: 31.60%, Eval Loss: 3.978391
Epoch 4 Train Loss: 3.802157, Eval Accuracy: 44.62%, Eval Loss: 3.749610
Epoch 5 Train Loss: 3.574698, Eval Accuracy: 48.72%, Eval Loss: 3.571569
Epoch 6 Train Loss: 3.369551, Eval Accuracy: 47.59%, Eval Loss: 3.400626
Epoch 7 Train Loss: 3.186699, Eval Accuracy: 49.02%, Eval Loss: 3.251072
Epoch 8 Train Loss: 2.999997, Eval Accuracy: 54.34%, Eval Loss: 3.101571
Epoch 9 Train Loss: 2.876097, Eval Accuracy: 55.20%, Eval Loss: 2.959369
Epoch 10 Train Loss: 2.668106, Eval Accuracy: 55.37%, Eval Loss: 2.853476
Epoch 11 Train Loss: 2.588547, Eval Accuracy: 58.12%, Eval Loss: 2.717815
Epoch 12 Train Loss: 2.405632, Eval Accuracy: 57.84%, Eval Loss: 2.642203
Epoch 13 Train Loss: 2.324905, Eval Accuracy: 59.61%, Eval Loss: 2.546059
Epoch 14 Train Loss: 2.254833, Eval

In [17]:
efficientnet_l_model = create_efficientnet_model('l')

print("EFFICIENTNET")

EN_train_acc, EN_rain_loss, EN_eval_acc, EN_eval_loss, EN_test_acc, EN_test_loss = train_eval_test(
    efficientnet_l_model, 
    train_data_loader, 
    val_data_loader, 
    test_data_loader,
    no_epochs=no_epochs
)

EFFICIENTNET
Epoch 1 Train Loss: 4.590027, Eval Accuracy: 15.59%, Eval Loss: 4.096957
Epoch 2 Train Loss: 3.842896, Eval Accuracy: 39.08%, Eval Loss: 3.421711
Epoch 3 Train Loss: 3.141942, Eval Accuracy: 63.46%, Eval Loss: 2.797723
Epoch 4 Train Loss: 2.599770, Eval Accuracy: 70.60%, Eval Loss: 2.338389
Epoch 5 Train Loss: 2.141581, Eval Accuracy: 76.46%, Eval Loss: 1.960250
Epoch 6 Train Loss: 1.845971, Eval Accuracy: 79.10%, Eval Loss: 1.681222
Epoch 7 Train Loss: 1.584890, Eval Accuracy: 83.33%, Eval Loss: 1.475755
Epoch 8 Train Loss: 1.366856, Eval Accuracy: 83.62%, Eval Loss: 1.310721
Epoch 9 Train Loss: 1.229933, Eval Accuracy: 84.31%, Eval Loss: 1.166734
Epoch 10 Train Loss: 1.117605, Eval Accuracy: 85.98%, Eval Loss: 1.079130
Epoch 11 Train Loss: 0.930863, Eval Accuracy: 86.49%, Eval Loss: 0.985865
Epoch 12 Train Loss: 0.943139, Eval Accuracy: 87.25%, Eval Loss: 0.923959
Epoch 13 Train Loss: 0.872363, Eval Accuracy: 89.13%, Eval Loss: 0.862135
Epoch 14 Train Loss: 0.771146, Eva